In [82]:
!pip install mysql-connector-python

In [83]:
#imporrts
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [84]:
#connsect with mysql
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="root123",
    database="restaurants",
    auth_plugin="mysql_native_password"
)
cursor = conn.cursor()
print("database connection established :D")

database connection established :D


In [85]:
#load data
orders = pd.read_csv("data/orders.csv")
print(f"Orders loaded: {len(orders)} records")
print(f"Columns: {orders.columns.tolist()}")
#orders.head()

users = pd.read_json("data/users.json")
print(f"Users loaded: {len(users)} records")
print(f"Columns: {users.columns.tolist()}")
#users.head()

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print(f"Restaurants loaded: {len(restaurants)} records")
print(f"Columns: {restaurants.columns.tolist()}")
#restaurants.head()

Orders loaded: 10000 records
Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']
Users loaded: 3000 records
Columns: ['user_id', 'name', 'city', 'membership']
Restaurants loaded: 500 records
Columns: ['restaurant_id', 'restaurant_name', 'cuisine', 'rating']


In [86]:
#data merge
orders_users = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)
print(f"Orders-Users merged: {len(orders_users)} records")
#orders_users.head()

final_df = pd.merge(
    orders_users,
    restaurants,
    how="left",
    on="restaurant_id"
)
print(f"Final dataset: {len(final_df)} records")
print(f"\nColumns: {final_df.columns.tolist()}")
#final_df.head()

Orders-Users merged: 10000 records
Final dataset: 10000 records

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']


In [87]:
#divide order datte
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)

final_df['quarter'] = final_df['order_date'].dt.quarter
final_df['month'] = final_df['order_date'].dt.month
final_df['year'] = final_df['order_date'].dt.year

print("dates standardised :D")

dates standardised :D


In [88]:
#alas
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
 12  quarter            10000 non-null  int32         
 13  month              10000 non-null  int32         
 14  year   

In [89]:
#save
final_df.to_csv("fnl_rest_ds.csv", index=False)
print("'fnl_rest_ds.csv' :D")

'fnl_rest_ds.csv' :D


In [90]:
#hue hue hue

In [91]:
#highest ttl revenue from gold members
final_df[final_df['membership'] == 'Gold']\
    .groupby('city')['total_amount']\
    .sum()\
    .sort_values(ascending=False)\
    .to_frame()

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [92]:
#cuisine w highest avg order val
final_df.groupby('cuisine')['total_amount']\
    .mean()\
    .sort_values(ascending=False)\
    .to_frame()

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [93]:
#distiict orders woirth >1000
user_ttl = final_df.groupby('user_id')['total_amount'].sum()
u_gt100 = (user_ttl > 1000).sum()
u_gt100

np.int64(2544)

In [94]:
#rest rating hi rev
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

final_df.groupby('rating_range')['total_amount']\
    .sum()\
    .sort_values(ascending=False)\
    .to_frame()

,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [95]:
#gold highest scity
final_df[final_df['membership'] == 'Gold']\
    .groupby('city')['total_amount']\
    .mean()\
    .sort_values(ascending=False)\
    .to_frame()

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [96]:
#cuisine lowest distinct restaurants
cuisine_stats = final_df\
    .groupby('cuisine')\
    .agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
    })\
    .rename(columns={'restaurant_id': 'num_restaurants', 'total_amount': 'total_revenue'})

cuisine_stats.sort_values('num_restaurants')

,num_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [97]:
#gold order perc
gold_orders = (final_df['membership'] == 'Gold').sum()
total_orders = len(final_df)
gold_perc = (gold_orders / total_orders) * 100
round(gold_perc)

50

In [98]:
#restaurant hi avg lt20
options = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian', 
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

for option in options:
    match = restaurant_stats[
        (restaurant_stats['restaurant_name'].str.contains(option, case=False, na=False)) &
        (restaurant_stats['total_orders'] < 20)
    ]
    
    if len(match) > 0:
        row = match.iloc[0]
        print(f"{option}: [cost: {row['avg_order_value']:.2f} // orders: {int(row['total_orders'])}]")

Ruchi Foods Chinese: [cost: 686.60 // orders: 19]


In [99]:
#cuisine/membership combo
final_df.groupby(['membership', 'cuisine'])['total_amount']\
    .sum()\
    .sort_values(ascending=False)\
    .to_frame()

total_amount
membership cuisine              
Regular    Mexican    1072943.30
           Italian    1018424.75
Gold       Mexican    1012559.79
           Italian    1005779.05
Regular    Indian      992100.27
Gold       Indian      979312.31
           Chinese     977713.74
Regular    Chinese     952790.91

In [100]:
#hightest quarter
final_df.groupby('quarter')['total_amount']\
    .sum()\
    .sort_values(ascending=False)\
    .to_frame()

,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [101]:
#ttl orders by gold
final_df[final_df['membership'] == 'Gold'].shape[0]

4987

In [102]:
#ttl rev frn hyd
round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())

1889367

In [103]:
#uniwq users w 1+ order
final_df['user_id'].nunique()

2883

In [104]:
#avg ordr val for gold
round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(),2)

np.float64(797.15)

In [105]:
#rest with >4.5 rating
final_df[final_df['rating'] >= 4.5].shape[0]

3374

In [106]:
#orders frm top rev city amg gold only
top_rev_city = (final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax())

final_df[(final_df['membership'] == 'Gold') & (final_df['city'] == top_rev_city)].shape[0]

1337